In [1]:
from dotenv import load_dotenv
import os
from qdrant_client import QdrantClient

# load variables from backend/.env
load_dotenv(dotenv_path=".env")

QDRANT_URL = os.environ["QDRANT_URL"]
QDRANT_API_KEY = os.environ["QDRANT_API_KEY"]
COLLECTION = os.environ.get("QDRANT_COLLECTION", "sda_dev_documentation")

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
count = client.count(collection_name=COLLECTION).count
print(f"Connected to: {QDRANT_URL}")
print(f"Collection: {COLLECTION}")
print(f"Total points indexed: {count}")


Connected to: https://4899ace7-5415-4b9e-911c-bed209888e4a.europe-west3-0.gcp.cloud.qdrant.io
Collection: sda_dev_documentation
Total points indexed: 22


In [2]:
result, next_page = client.scroll(
    collection_name=COLLECTION,
    limit=5,
    with_payload=True,
    with_vectors=False
)

for i, item in enumerate(result, 1):
    p = item.payload
    print(f"--- Chunk {i} ---")
    print("Source:", p.get("source"))
    print("Doctype:", p.get("doctype"))
    print("Chunk ID:", p.get("chunk_id"))
    print("Preview:", p.get("text", "")[:250].replace("\n", " "), "\n")


--- Chunk 1 ---
Source: api_service.txt
Doctype: None
Chunk ID: 0
Preview: The internal API Service runs on port 8080. Authentication requires an 'X-Internal-Key' header set to 'super-secret-key-prod'. The main endpoint for fetching user data is /user/details/{id}. 

--- Chunk 2 ---
Source: frontend_styling.txt
Doctype: None
Chunk ID: 0
Preview: All new React components must use Tailwind CSS for styling. The primary color is blue-600. Do not use Material UI components unless displaying large grid data. 

--- Chunk 3 ---
Source: README.md
Doctype: None
Chunk ID: 5
Preview: ---  ## 📜 License  This project is licensed under the [MIT License](./LICENSE.md).  ---  ## 🙏 Acknowledgements  This project uses the following open-source libraries and frameworks:  - [LangChain](https://github.com/langchain-ai/langchain) — MIT Lice 

--- Chunk 4 ---
Source: TECHNICAL_DESIGN.md
Doctype: None
Chunk ID: 5
Preview: ---  ## 7. Roadmap  - **v1.0.0 Release:** Tag Phase 2 as stable.   - **Phase 3:** Add use

In [4]:
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# vs = Qdrant(client=client, collection_name=COLLECTION, embeddings=embeddings)
vs = Qdrant(
    client=client,
    collection_name=COLLECTION,
    embeddings=embeddings,
    content_payload_key="text"   # 👈 match your payload key
)

docs = vs.similarity_search("What port does the internal API run on?", k=3)
for d in docs:
    print(d.page_content[:250], "\n---\n")


The internal API Service runs on port 8080.
Authentication requires an 'X-Internal-Key' header set to 'super-secret-key-prod'.
The main endpoint for fetching user data is /user/details/{id}. 
---

The internal API Service runs on port 8080.
Authentication requires an 'X-Internal-Key' header set to 'super-secret-key-prod'.
The main endpoint for fetching user data is /user/details/{id}. 
---

---

## 3. Backend Implementation Details

### 🧩 Backend Component Architecture

The backend is modularly designed with FastAPI serving as the API layer, LangChain handling LLM orchestration, and Qdrant/PostgreSQL as the data stores.

![Backend Archi 
---



In [7]:
import requests

resp = requests.post(
    "http://localhost:8000/api/v1/answer_from_docs",
    json={"prompt_text": "What port does the internal API run on?", "top_k": 6, "rerank": True}
)
print(resp.json()["generated_content"])


The internal API Service runs on port 8080.


In [8]:
import requests
resp = requests.post(
    "http://localhost:8000/api/v1/answer_from_docs",
    json={
        "prompt_text": "Which authentication header is required by the internal API?",
        "top_k": 6,
        "rerank": True
    }
)
print(resp.json()["generated_content"])


The internal API requires the 'X-Internal-Key' header to be set to 'super-secret-key-prod' for authentication.
